# Evaluation of tsdr on localization

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

import logging
logger = logging.getLogger()
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

In [5]:
SOCKSHOP_DATASET_ID = "9n6mf"
TRAINTICKET_DATASET_ID = "m9dgg"

from meltria.priorknowledge.priorknowledge import SockShopKnowledge

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewares": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

In [6]:
from notebooklib import localization

results = localization.grid_dataset([
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.FIRST),  # skip phase2
], TRAINTICKET_DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results:
    display(cache_suffix, score_df)

Loading m9dgg with hierarchy-single_sbd_medoid_only_ctnrs...
Processing m9dgg with hierarchy-single_sbd_medoid_only_ctnrs...


No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


KeyError: 'ts-delivery'

In [1]:
from notebooklib import localization

results = localization.grid_dataset([
    ("hdbscan_sbd_only_ctnrs", localization.DiagTargetPhaseOption.FIRST),  # skip phase2
], SOCKSHOP_DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results:
    display(cache_suffix, score_df)

ModuleNotFoundError: No module named 'eval'

In [13]:
from notebooklib import localization


results = localization.grid_dataset([
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.FIRST),  # skip phase2
    "hierarchy-single_sbd_medoid_only_ctnrs",
    "hdbscan_sbd_only_ctnrs",
    "fluxrank_pearsonr_medoid_only_ctnrs",
    # "fluxrank_pearsonr_max_cluster_only_ctnrs",
    "fluxrank_sbd_medoid_only_ctnrs",
    # "hdbscan_sbd_max_cluster_only_ctnrs",
    "phase2only_hierarchy-single_sbd_medoid_only_ctnrs",
    "phase1only_kstest_only_ctnrs",
    "phase1only_ediagnosis_only_ctnrs",
    "phase1only_fluxinfer_only_ctnrs",
], SOCKSHOP_DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.20,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results:
    display(cache_suffix, score_df)

Loading 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...
Processing 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...


No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause met

...

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,34,0.441176,0.441176,105,0.180952,0.180952,105,0.200000,0.200000
2,34,0.647059,0.544118,105,0.361905,0.271429,105,0.409524,0.304762
3,34,0.735294,0.607843,105,0.457143,0.333333,105,0.504762,0.371429
4,34,0.852941,0.669118,105,0.552381,0.388095,105,0.647619,0.440476
5,34,0.911765,0.717647,105,0.571429,0.424762,105,0.800000,0.512381
6,34,0.941176,0.754902,105,0.600000,0.453968,105,0.857143,0.569841
7,34,0.970588,0.785714,105,0.609524,0.476190,105,0.866667,0.612245
8,34,1.000000,0.812500,105,0.609524,0.492857,105,0.866667,0.644048
9,34,1.000000,0.833333,105,0.609524,0.505820,105,0.866667,0.668783
10,34,1.000000,0.850000,105,0.609524,0.516190,105,0.866667,0.688571


('phase1only_ediagnosis_only_ctnrs', <DiagTargetPhaseOption.LAST: -1>)

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,39,0.410256,0.410256,105,0.142857,0.142857,105,0.209524,0.209524
2,39,0.538462,0.474359,105,0.295238,0.219048,105,0.447619,0.328571
3,39,0.717949,0.555556,105,0.466667,0.301587,105,0.552381,0.403175
4,39,0.820513,0.621795,105,0.523810,0.357143,105,0.685714,0.473810
5,39,0.871795,0.671795,105,0.590476,0.403810,105,0.857143,0.550476
6,39,0.897436,0.709402,105,0.619048,0.439683,105,0.904762,0.609524
7,39,0.974359,0.747253,105,0.619048,0.465306,105,0.914286,0.653061
8,39,1.000000,0.778846,105,0.619048,0.484524,105,0.914286,0.685714
9,39,1.000000,0.803419,105,0.619048,0.499471,105,0.914286,0.711111
10,39,1.000000,0.823077,105,0.619048,0.511429,105,0.914286,0.731429


('phase1only_fluxinfer_only_ctnrs', <DiagTargetPhaseOption.LAST: -1>)

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,44,0.409091,0.409091,105,0.180952,0.180952,105,0.133333,0.133333
2,44,0.659091,0.534091,105,0.361905,0.271429,105,0.438095,0.285714
3,44,0.750000,0.606061,105,0.495238,0.346032,105,0.571429,0.380952
4,44,0.840909,0.664773,105,0.571429,0.402381,105,0.695238,0.459524
5,44,0.909091,0.713636,105,0.600000,0.441905,105,0.847619,0.537143
6,44,0.931818,0.750000,105,0.628571,0.473016,105,0.885714,0.595238
7,44,0.977273,0.782468,105,0.628571,0.495238,105,0.895238,0.638095
8,44,1.000000,0.809659,105,0.628571,0.511905,105,0.895238,0.670238
9,44,1.000000,0.830808,105,0.628571,0.524868,105,0.895238,0.695238
10,44,1.000000,0.847727,105,0.628571,0.535238,105,0.895238,0.715238


In [15]:
results2 = localization.grid_dataset([
    ("hierarchy-single_sbd_medoid_only_ctnrs", localization.DiagTargetPhaseOption.FIRST),  # skip phase2
    "hierarchy-single_sbd_medoid_only_ctnrs",
    "hdbscan_sbd_only_ctnrs",
    "fluxrank_pearsonr_medoid_only_ctnrs",
    # "fluxrank_pearsonr_max_cluster_only_ctnrs",
    "fluxrank_sbd_medoid_only_ctnrs",
    # "hdbscan_sbd_max_cluster_only_ctnrs",
    "phase2only_hierarchy-single_sbd_medoid_only_ctnrs",
    "phase1only_kstest_only_ctnrs",
    "phase1only_ediagnosis_only_ctnrs",
    "phase1only_fluxinfer_only_ctnrs",
], SOCKSHOP_DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=True,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.10,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results2:
    display(cache_suffix, score_df)

Loading 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...
Processing 9n6mf with hierarchy-single_sbd_medoid_only_ctnrs...


No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, soc

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 

...

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,30,0.233333,0.233333,105,0.104762,0.104762,105,0.228571,0.228571
2,30,0.366667,0.300000,105,0.266667,0.185714,105,0.466667,0.347619
3,30,0.566667,0.388889,105,0.352381,0.241270,105,0.533333,0.409524
4,30,0.733333,0.475000,105,0.419048,0.285714,105,0.657143,0.471429
5,30,0.766667,0.533333,105,0.447619,0.318095,105,0.761905,0.529524
6,30,0.866667,0.588889,105,0.447619,0.339683,105,0.780952,0.571429
7,30,0.966667,0.642857,105,0.447619,0.355102,105,0.780952,0.601361
8,30,1.000000,0.687500,105,0.447619,0.366667,105,0.780952,0.623810
9,30,1.000000,0.722222,105,0.447619,0.375661,105,0.780952,0.641270
10,30,1.000000,0.750000,105,0.447619,0.382857,105,0.780952,0.655238


('phase1only_ediagnosis_only_ctnrs', <DiagTargetPhaseOption.LAST: -1>)

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,28,0.357143,0.357143,105,0.152381,0.152381,105,0.180952,0.180952
2,28,0.464286,0.410714,105,0.304762,0.228571,105,0.466667,0.323810
3,28,0.678571,0.500000,105,0.409524,0.288889,105,0.552381,0.400000
4,28,0.857143,0.589286,105,0.457143,0.330952,105,0.628571,0.457143
5,28,0.928571,0.657143,105,0.466667,0.358095,105,0.733333,0.512381
6,28,1.000000,0.714286,105,0.485714,0.379365,105,0.742857,0.550794
7,28,1.000000,0.755102,105,0.485714,0.394558,105,0.742857,0.578231
8,28,1.000000,0.785714,105,0.485714,0.405952,105,0.742857,0.598810
9,28,1.000000,0.809524,105,0.485714,0.414815,105,0.742857,0.614815
10,28,1.000000,0.828571,105,0.485714,0.421905,105,0.742857,0.627619


('phase1only_fluxinfer_only_ctnrs', <DiagTargetPhaseOption.LAST: -1>)

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,33,0.424242,0.424242,105,0.161905,0.161905,105,0.209524,0.209524
2,33,0.545455,0.484848,105,0.276190,0.219048,105,0.419048,0.314286
3,33,0.666667,0.545455,105,0.428571,0.288889,105,0.580952,0.403175
4,33,0.757576,0.598485,105,0.514286,0.345238,105,0.666667,0.469048
5,33,0.787879,0.636364,105,0.533333,0.382857,105,0.752381,0.525714
6,33,0.878788,0.676768,105,0.533333,0.407937,105,0.790476,0.569841
7,33,0.878788,0.705628,105,0.533333,0.425850,105,0.790476,0.601361
8,33,0.939394,0.734848,105,0.533333,0.439286,105,0.790476,0.625000
9,33,0.939394,0.757576,105,0.533333,0.449735,105,0.790476,0.643386
10,33,1.000000,0.781818,105,0.533333,0.458095,105,0.790476,0.658095
